In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

response = requests.get("https://www.tnwb.uscourts.gov/Search/search.aspx?zoom_query=car&zoom_page=1&zoom_per_page=132&zoom_and=1&zoom_sort=0&zoom_xml=0")
doc = BeautifulSoup(response.text)

In [41]:
items = doc.find_all(True, {"class":["result_block", "result_altblock"]})

rows = []

for item in items:

    row = {}
    #To get the numbers
    row['number'] = item.select_one('.result_title b').text.strip()
    #To get the case name
    row['case_name'] = item.select_one('.result_title a').text.strip()
    #To get the url
    row ['url'] = item.select_one('.result_title a').get('href')
    #To get category
    row['category'] = item.select_one('.category').text.strip().replace('[','').replace(']','')
    #To get terms matched
    terms = item.select_one('.infoline').text.split('-')[1].strip()
    #To get context
    context = item.select_one('.context').text.strip().replace('...','')
    #combining rows
    row['additional_details'] = f'{terms} - {context}'

    rows.append(row)

In [42]:
df = pd.json_normalize(rows)
df.head(10)

,number,case_name,url,category,additional_details
0,1.,JDL: 04-24318 Jacquelline D. Black,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,Judges' Opinions,102k - the basis that the Debtor failed to pr...
1,2.,WHB: 95-26401 Mary Lucy Cooper,https://www.tnwb.uscourts.gov/Opinions/whb/pdf...,Judges' Opinions,"27k - MARY LUCY COOPER, Plaintiff, v. Adversa..."
2,3.,GHB: 97-12368 Billy G. Woffard,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,Judges' Opinions,"71k - G. Woffard, ("" Woffard"") , was partners..."
3,4.,JDL: 97-30580 Mary Chrlis Hurst,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,Judges' Opinions,32k - UNITED STATES BANKRUPTCY COURT WESTERN ...
4,5.,MRH: 20-20967 Jacob Braxton Herring 20-00094,https://www.tnwb.uscourts.gov/Opinions/mrh/pdf...,Judges' Opinions,303k - and soon thereafter the contract was a...
5,6.,GHB: 95-11365 Melissa L. Bryan,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,Judges' Opinions,"52k - ""( a ) A discharge under section 1 The ..."
6,7.,JDL: 09-20339 Diane M. Miller,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,Judges' Opinions,92k - vehicle. The vehicle was damaged in an ...
7,8.,GHB: 00-12340 Wanda K. Autry,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,Judges' Opinions,"297k - $1,800.82 in full, the debtor asked th..."
8,9.,"GHB: 02-31651 Neil Bond Stewart, Jr. and Tina ...",https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,Judges' Opinions,"291k - . STEW ART , Case No. 02-31651 Debtors..."
9,10.,GHB: 96-12039 Randy and Janice Willson,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,Judges' Opinions,201k - re Union Planters' Motion to Compel De...


In [43]:
df.to_csv("car_records.csv", index=False)